In [1]:
from __future__ import print_function

# Import libraries
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties # for unicode fonts
import psycopg2
import sys
import datetime as dt
import mp_utils as mp

from collections import OrderedDict

# used to print out pretty pandas dataframes
from IPython.display import display, HTML

from sklearn.pipeline import Pipeline

# used to impute mean for data and standardize for computational stability
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# logistic regression is our favourite model ever
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV # l2 regularized regression
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier

# used to calculate AUROC/accuracy
from sklearn import metrics

# used to create confusion matrix
from sklearn.metrics import confusion_matrix

# gradient boosting - must download package https://github.com/dmlc/xgboost
import xgboost as xgb

%matplotlib inline

# below config used on pc70
sqluser = 'alistairewj'
dbname = 'mimic'
schema_name = 'mimiciii'
query_schema = 'SET search_path to public,' + schema_name + ';'

In [ ]:
USE_SQL=0

if USE_SQL:
    # Connect to local postgres version of mimic
    con = psycopg2.connect(dbname=dbname, user=sqluser)

    # exclusion criteria:
    #   - less than 15 years old
    #   - stayed in the ICU less than 4 hours
    #   - never have any chartevents data (i.e. likely administrative error)
    #   - organ donor accounts (administrative "readmissions" for patients who died in hospital)
    query = query_schema + \
    """
    select 
        *
    from dm_cohort
    where excluded = 0
    """
    co = pd.read_sql_query(query,con)
    
    # convert the inclusion flags to boolean
    for c in co.columns:
        if c[0:10]=='inclusion_':
            co[c] = co[c].astype(bool)

    # extract static vars into a separate dataframe
    df_static = pd.read_sql_query(query_schema + 'select * from mp_static_data', con)
    #for dtvar in ['intime','outtime','deathtime']:
    #    df_static[dtvar] = pd.to_datetime(df_static[dtvar])

    vars_static = [u'is_male', u'emergency_admission', u'age',
                   # services
                   u'service_any_noncard_surg',
                   u'service_any_card_surg',
                   u'service_cmed',
                   u'service_traum',
                   u'service_nmed',
                   # ethnicities
                   u'race_black',u'race_hispanic',u'race_asian',u'race_other',
                   # phatness
                   u'height', u'weight', u'bmi']


    # get ~5 million rows containing data from errbody
    # this takes a little bit of time to load into memory (~2 minutes)

    # %%time results
    # CPU times: user 42.8 s, sys: 1min 3s, total: 1min 46s
    # Wall time: 2min 7s

    df = pd.read_sql_query(query_schema + 'select * from mp_data', con)
    df.drop('subject_id',axis=1,inplace=True)
    df.drop('hadm_id',axis=1,inplace=True)
    df.sort_values(['icustay_id','hr'],axis=0,ascending=True,inplace=True)
    print(df.shape)

    # get death information
    df_death = pd.read_sql_query(query_schema + """
    select 
    co.subject_id, co.hadm_id, co.icustay_id
    , ceil(extract(epoch from (co.outtime - co.intime))/60.0/60.0) as dischtime_hours
    , ceil(extract(epoch from (adm.deathtime - co.intime))/60.0/60.0) as deathtime_hours
    , case when adm.deathtime is null then 0 else 1 end as death
    from dm_cohort co
    inner join admissions adm
    on co.hadm_id = adm.hadm_id
    where co.excluded = 0
    """, con)
    
    # get censoring information
    df_censor = pd.read_sql_query(query_schema + """
    select co.icustay_id, min(cs.charttime) as censortime
    , ceil(extract(epoch from min(cs.charttime-co.intime) )/60.0/60.0) as censortime_hours
    from dm_cohort co 
    inner join mp_code_status cs
    on co.icustay_id = cs.icustay_id
    where cmo+dnr+dni+dncpr+cmo_notes>0
    and co.excluded = 0
    group by co.icustay_id
    """, con)

    # get severity scores
    df_soi = pd.read_sql_query(query_schema + """
    select 
    co.icustay_id
    , case when adm.deathtime is null then 0 else 1 end as death
    , sa.saps
    , sa2.sapsii
    , aps.apsiii
    , so.sofa
    , lo.lods
    , oa.oasis
    from dm_cohort co
    inner join admissions adm
    on co.hadm_id = adm.hadm_id
    left join saps sa
    on co.icustay_id = sa.icustay_id
    left join sapsii sa2
    on co.icustay_id = sa2.icustay_id
    left join apsiii aps
    on co.icustay_id = aps.icustay_id
    left join sofa so
    on co.icustay_id = so.icustay_id
    left join lods lo
    on co.icustay_id = lo.icustay_id
    left join oasis oa
    on co.icustay_id = oa.icustay_id
    where co.excluded = 0
    """, con)
    
    # extract static vars into a separate dataframe
    df_static = pd.read_sql_query(query_schema + 'select * from mp_static_data', con)

In [34]:
USE_CSV=1

if USE_CSV:
    co = pd.read_csv('df_cohort.csv')
    
    # convert the inclusion flags to boolean
    for c in co.columns:
        if c[0:10]=='inclusion_':
            co[c] = co[c].astype(bool)
    df = pd.read_csv('df_data.csv')
    df_static = pd.read_csv('df_static_data.csv')
    df_censor = pd.read_csv('df_censor.csv')
    df_death = pd.read_csv('df_death.csv')
    df_soi = pd.read_csv('df_soi.csv')

# Exclusion criteria

Each study has its own exclusion criteria (sometimes studies have multiple experiments). We define a dictionary of all exclusions with the dictionary key as the study name. Some studies have multiple experiments, so we append *a*, *b*, or *c*.

The dictionary stores a length 2 list. The first element defines the window for data extraction: it contains a dictionary of the windows and the corresponding window sizes. The second element is the exclusion criteria. Both are functions which use `co` or `df` as their input.

In [26]:
# first we can define the different windows: there aren't that many!
df_tmp=df_death.copy().merge(df_censor, how='left', left_on='icustay_id', right_on='icustay_id').set_index('icustay_id')

# admission+12 hours
time_12hr = df_tmp.copy()
time_12hr['windowtime'] = 12
time_12hr = time_12hr['windowtime'].to_dict()

# admission+24 hours
time_24hr = df_tmp.copy()
time_24hr['windowtime'] = 24
time_24hr = time_24hr['windowtime'].to_dict()

# admission+48 hours
time_48hr = df_tmp.copy()
time_48hr['windowtime'] = 48
time_48hr = time_48hr['windowtime'].to_dict()

# admission+72 hours
time_72hr = df_tmp.copy()
time_72hr['windowtime'] = 72
time_72hr = time_72hr['windowtime'].to_dict()

# admission+96 hours
time_96hr = df_tmp.copy()
time_96hr['windowtime'] = 96
time_96hr = time_96hr['windowtime'].to_dict()

# entire stay
time_all = df_tmp.copy()
time_all = time_all['dischtime_hours'].apply(np.ceil).astype(int).to_dict()

# 12 hours before the patient died/discharged
time_predeath = df_tmp.copy()
time_predeath['windowtime'] = time_predeath['dischtime_hours']
idx = time_predeath['deathtime_hours']<time_predeath['dischtime_hours']
time_predeath.loc[idx,'windowtime'] = time_predeath.loc[idx,'deathtime_hours']
# move from discharge/death time to 12 hours beforehand
time_predeath['windowtime'] = time_predeath['windowtime']-12
time_predeath = time_predeath['windowtime'].apply(np.ceil).astype(int).to_dict()

In [41]:
# example params used to extract patient data
# element 1: dictionary specifying end time of window for each patient
# element 2: size of window
# element 3: extra hours added to make it easier to get data on labs (and allows us to get labs pre-ICU)
# e.g. [time_24hr, 8, 24] is
#   (1) window ends at admission+24hr
#   (2) window is 8 hours long
#   (3) lab window is 8+24=32 hours long

# this one is used more than once, so we define it here
hugExclFcn = lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_hug2009_obs']&x['inclusion_hug2009_not_nsicu_csicu']&x['inclusion_first_admission']&x['inclusion_full_code']&x['inclusion_not_brain_death']&x['inclusion_not_crf']&x['inclusion_no_dialysis_first24hr'],'icustay_id'].values

# physionet2012 subset - not exact but close
def physChallExclFcn(x):
    out = x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_stay_ge_48hr']&x['inclusion_has_saps'],'icustay_id'].values
    out = np.sort(out)
    out = out[0:4000]
    return out
 
# caballero2015 is a random subsample - then limits to 18yrs, resulting in 11648
def caballeroExclFcn(x):
    out = x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18'],'icustay_id'].values
    out = np.sort(out)
    out = out[0:11648]
    return out

np.random.seed(546345)
W_extra = 24

exclusions = OrderedDict([
['caballero2015dynamically_a',  [[time_24hr, 24, W_extra], caballeroExclFcn]],
['caballero2015dynamically_b',  [[time_48hr, 48, W_extra], caballeroExclFcn]],
['caballero2015dynamically_c',  [[time_72hr, 72, W_extra], caballeroExclFcn]],
['calvert2016computational',    [[time_predeath, 5, W_extra], lambda x: x.loc[x['inclusion_over_18']&x['inclusion_calvert2016_obs']&x['inclusion_stay_ge_17hr']&x['inclusion_non_alc_icd9'],'icustay_id'].values]],
['calvert2016using',            [[time_predeath, 5, W_extra], lambda x: x.loc[x['inclusion_over_18']&x['inclusion_calvert2016_obs']&x['inclusion_stay_ge_17hr']&x['inclusion_stay_le_500hr'],'icustay_id'].values]],
['celi2012database_a',          [[time_72hr, 72, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_aki_icd9'],'icustay_id'].values ]],
['celi2012database_b',          [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_sah_icd9'],'icustay_id'].values ]],
['che2016recurrent_a',          [[time_48hr, 48, W_extra], lambda x: x.loc[x['inclusion_over_18'],'icustay_id'].values ]],
['che2016recurrent_b',          [[time_48hr, 48, W_extra], physChallExclFcn ]],
['ding2016mortality',           [[time_48hr, 48, W_extra], physChallExclFcn ]],
['ghassemi2014unfolding_a',     [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_ge_100_non_stop_words'],'icustay_id'].values]],
['ghassemi2014unfolding_b',     [[time_12hr, 12, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_ge_100_non_stop_words'],'icustay_id'].values]],
['ghassemi2015multivariate',    [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_ge_100_non_stop_words']&x['inclusion_gt_6_notes'],'icustay_id'].values]],
['grnarova2016neural',          [[time_all,  24, W_extra], lambda x: x.loc[x['inclusion_over_18']&x['inclusion_multiple_hadm'],'icustay_id'].values]],
['harutyunyan2017multitask',    [[time_48hr, 48, W_extra], lambda x: x.loc[x['inclusion_over_18']&x['inclusion_multiple_icustay'],'icustay_id'].values]],
['hoogendoorn2016prediction',   [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_hug2009_obs'],'icustay_id'].values]],
['hug2009icu',                  [[time_24hr, 24, W_extra], hugExclFcn]],
['johnson2012patient',          [[time_48hr, 48, W_extra], physChallExclFcn]],
['johnson2014data',             [[time_48hr, 48, W_extra], physChallExclFcn]],
['joshi2012prognostic',         [[time_24hr, 24, W_extra], hugExclFcn]],
['joshi2016identifiable',       [[time_48hr, 48, W_extra], hugExclFcn]],
['lee2015customization',        [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_lee2015_service']&x['inclusion_has_saps'],'icustay_id'].values]],
['lee2015personalized',         [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_has_saps'],'icustay_id'].values]],
['lee2017patient',              [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_has_saps'],'icustay_id'].values]],
['lehman2012risk',              [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_has_saps']&x['inclusion_stay_ge_24hr']&x['inclusion_first_admission'],'icustay_id'].values]],
['luo2016interpretable',        [[time_all,  24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_has_sapsii']&x['inclusion_no_disch_summary'],'icustay_id'].values]],
['luo2016predicting',           [[time_24hr, 12, W_extra], lambda x: np.intersect1d(hugExclFcn(x),x.loc[x['inclusion_stay_ge_24hr'],'icustay_id'].values) ]],
['pirracchio2015mortality',     [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_15'],'icustay_id'].values ]],
['ripoll2014sepsis',            [[time_24hr, 24, W_extra], lambda x: x.loc[x['inclusion_only_mimicii']&x['inclusion_over_18']&x['inclusion_has_saps']&x['inclusion_not_explicit_sepsis'],'icustay_id'].values]],
['wojtusiak2017c',              [[time_all,  24, W_extra], lambda x: x.loc[x['inclusion_over_65']&x['inclusion_alive_hos_disch'],'icustay_id'].values]]
])

In [43]:
# define var_static which is used later
#TODO: should refactor so this isn't needed
var_min, var_max, var_first, var_last, var_sum, var_first_early, var_last_early, var_static = mp.vars_of_interest()

K=5
np.random.seed(871)
# get unique subject_id (this is needed later)
sid = np.sort(np.unique(df_death['subject_id'].values))

# assign k-fold
idxK_sid = np.random.permutation(sid.shape[0])
idxK_sid = np.mod(idxK_sid,K)

# get indices which map subject_ids in sid to the X dataframe
idxMap = np.searchsorted(sid, df_death['subject_id'].values)

# use these indices to map the k-fold integers
idxK = idxK_sid[idxMap]

In [44]:
mdl_val_all = dict()
results_val_all = dict()
pred_val_all = dict()
tar_val_all = dict()

# Rough timing info:
#     rf - 3 seconds per fold
#    xgb - 30 seconds per fold
# logreg - 4 seconds per fold
#  lasso - 8 seconds per fold
models = OrderedDict([
          ['xgb', xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)],
          ['lasso', LassoCV(cv=5,fit_intercept=True,normalize=True,max_iter=10000)],
          ['logreg', LogisticRegression(fit_intercept=True)],
          ['rf', RandomForestClassifier()]
         ])
#models = OrderedDict([['xgb', xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)]])
    
with open('/data/Dropbox (MIT)/data_matters_results/results.txt','w') as fp:
    fp.write('StudyName,SampleSize')
    for mdl in models:
        fp.write(',{}'.format(mdl))
    fp.write('\n')
    
for current_study in exclusions:    
    print('\n==================== {} =========='.format('='*len(current_study)))
    print('========== BEGINNING {} =========='.format(current_study))
    print('==================== {} =========='.format('='*len(current_study)))

    params = exclusions[current_study][0]
    df_data = mp.get_design_matrix(df, params[0], W=params[1], W_extra=params[2])

    # get a list of icustay_id who stayed at least 12 hours
    iid_keep = exclusions[current_study][1](co)
    print('Reducing sample size from {} to {} ({:2.2f}%).'.format(
            df_data.shape[0], iid_keep.shape[0], iid_keep.shape[0]*100.0 / df_data.shape[0]))
    df_data = df_data.loc[iid_keep,:]
    print('')

    # load the data into a numpy array

    # first, the data from static vars from df_static
    X = df_data.merge(df_static.set_index('icustay_id')[var_static], how='left', left_index=True, right_index=True)
    # next, add in the outcome: death in hospital
    X = X.merge(df_death.set_index('icustay_id')[['death']], left_index=True, right_index=True)

    # map above K-fold indices to this dataset
    X = X.merge(df_death.set_index('icustay_id')[['subject_id']], left_index=True, right_index=True)
    # get indices which map subject_ids in sid to the X dataframe
    idxMap = np.searchsorted(sid, X['subject_id'].values)
    # use these indices to map the k-fold integers
    idxK = idxK_sid[idxMap]
    # drop the subject_id column
    X.drop('subject_id',axis=1,inplace=True)

    # convert to numpy data (assumes target, death, is the last column)
    X = X.values
    y = X[:,-1]
    X = X[:,0:-1]
    X_header = [x for x in df_data.columns.values] + var_static
    
    mdl_val = dict()
    results_val = dict()
    pred_val = dict()
    tar_val = dict()

    for mdl in models:
        print('=============== {} ==============='.format(mdl))
        mdl_val[mdl] = list()
        results_val[mdl] = list() # initialize list for scores
        pred_val[mdl] = list()
        tar_val[mdl] = list()

        if mdl == 'xgb':
            # no pre-processing of data necessary for xgb
            estimator = Pipeline([(mdl, models[mdl])])

        else:
            estimator = Pipeline([("imputer", Imputer(missing_values='NaN',
                                              strategy="mean",
                                              axis=0)),
                          ("scaler", StandardScaler()),
                          (mdl, models[mdl])]) 

        for k in range(K):
            # train the model using all but the kth fold
            curr_mdl = estimator.fit(X[idxK != k, :],y[idxK != k])

            # get prediction on this dataset
            if mdl == 'lasso':
                curr_prob = curr_mdl.predict(X[idxK == k, :])
            else:
                curr_prob = curr_mdl.predict_proba(X[idxK == k, :])
                curr_prob = curr_prob[:,1]

            pred_val[mdl].append(curr_prob)
            tar_val[mdl].append(y[idxK == k])

            # calculate score (AUROC)
            curr_score = metrics.roc_auc_score(y[idxK == k], curr_prob)

            # add score to list of scores
            results_val[mdl].append(curr_score)

            # save the current model
            mdl_val[mdl].append(curr_mdl)

            print('{} - Finished fold {} of {}. AUROC {:0.3f}.'.format(dt.datetime.now(), k+1, K, curr_score))

    # create a pointer for above dicts with new var names
    # we will likely re-use the dicts in subsequent calls for getting model perfomances
    mdl_val_all[current_study] = mdl_val
    results_val_all[current_study] = results_val
    pred_val_all[current_study] = pred_val
    tar_val_all[current_study] = tar_val
    
    # print to file
    with open('/data/Dropbox (MIT)/data_matters_results/results.txt','a') as fp:
        # print study name and sample size
        fp.write( '{},{}'.format(current_study, X.shape[0] ) )
        
        for i, mdl in enumerate(models):
            fp.write(',{:0.6f}'.format( np.mean(results_val[mdl]) ))
        
        fp.write('\n')

In [55]:
EXAMPLE_RUN=True

# Rough timing info:
#     rf - 3 seconds per fold
#    xgb - 30 seconds per fold
# logreg - 4 seconds per fold
#  lasso - 8 seconds per fold
models = OrderedDict([
          ['xgb', xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)],
          ['lasso', LassoCV(cv=5,fit_intercept=True,normalize=True,max_iter=10000)],
          ['logreg', LogisticRegression(fit_intercept=True)],
          ['rf', RandomForestClassifier()]
         ])
#models = OrderedDict([['xgb', xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)]])
    
if EXAMPLE_RUN:
    current_study = 'lehman2012risk'
    
    print('\n==================== {} =========='.format('='*len(current_study)))
    print('========== BEGINNING {} =========='.format(current_study))
    print('==================== {} =========='.format('='*len(current_study)))

    params = exclusions[current_study][0]
    df_data = mp.get_design_matrix(df, params[0], W=params[1], W_extra=params[2])

    # get a list of icustay_id who stayed at least 12 hours
    iid_keep = exclusions['lehman2012risk'][1](co)
    print('Reducing sample size from {} to {} ({:2.2f}%).'.format(
            df_data.shape[0], iid_keep.shape[0], iid_keep.shape[0]*100.0 / df_data.shape[0]))
    df_data = df_data.loc[iid_keep,:]
    print('')

    # load the data into a numpy array

    # first, the data from static vars from df_static
    X = df_data.merge(df_static.set_index('icustay_id')[var_static], how='left', left_index=True, right_index=True)
    # next, add in the outcome: death in hospital
    X = X.merge(df_death.set_index('icustay_id')[['death']], left_index=True, right_index=True)

    # map above K-fold indices to this dataset
    X = X.merge(df_death.set_index('icustay_id')[['subject_id']], left_index=True, right_index=True)
    # get indices which map subject_ids in sid to the X dataframe
    idxMap = np.searchsorted(sid, X['subject_id'].values)
    # use these indices to map the k-fold integers
    idxK = idxK_sid[idxMap]
    # drop the subject_id column
    X.drop('subject_id',axis=1,inplace=True)

    # convert to numpy data (assumes target, death, is the last column)
    X = X.values
    y = X[:,-1]
    X = X[:,0:-1]
    X_header = [x for x in df_data.columns.values] + var_static
    
    mdl_val = dict()
    results_val = dict()
    pred_val = dict()
    tar_val = dict()

    for mdl in models:
        print('=============== {} ==============='.format(mdl))
        mdl_val[mdl] = list()
        results_val[mdl] = list() # initialize list for scores
        pred_val[mdl] = list()
        tar_val[mdl] = list()

        if mdl == 'xgb':
            # no pre-processing of data necessary for xgb
            estimator = Pipeline([(mdl, models[mdl])])

        else:
            estimator = Pipeline([("imputer", Imputer(missing_values='NaN',
                                              strategy="mean",
                                              axis=0)),
                          ("scaler", StandardScaler()),
                          (mdl, models[mdl])]) 

        for k in range(K):
            # train the model using all but the kth fold
            curr_mdl = estimator.fit(X[idxK != k, :],y[idxK != k])

            # get prediction on this dataset
            if mdl == 'lasso':
                curr_prob = curr_mdl.predict(X[idxK == k, :])
            else:
                curr_prob = curr_mdl.predict_proba(X[idxK == k, :])
                curr_prob = curr_prob[:,1]

            pred_val[mdl].append(curr_prob)
            tar_val[mdl].append(y[idxK == k])

            # calculate score (AUROC)
            curr_score = metrics.roc_auc_score(y[idxK == k], curr_prob)

            # add score to list of scores
            results_val[mdl].append(curr_score)

            # save the current model
            mdl_val[mdl].append(curr_mdl)

            print('{} - Finished fold {} of {}. AUROC {:0.3f}.'.format(dt.datetime.now(), k+1, K, curr_score))


==================== ============== ==========
========== BEGINNING lehman2012risk ==========
==================== ============== ==========
Reducing sample size from 52050 to 21739 (41.77%).

=============== xgb ===============
2017-04-23 20:22:36.959390 - Finished fold 1 of 5. AUROC 0.877.
2017-04-23 20:22:45.229326 - Finished fold 2 of 5. AUROC 0.883.
2017-04-23 20:22:53.804337 - Finished fold 3 of 5. AUROC 0.891.
2017-04-23 20:23:02.824029 - Finished fold 4 of 5. AUROC 0.891.
2017-04-23 20:23:10.956716 - Finished fold 5 of 5. AUROC 0.897.


TypeError: unsupported operand type(s) for /: 'dict' and 'int'